In [1]:
import blankly
import os
from dotenv import load_dotenv
import pprint as pp

load_dotenv("./alpaca.env")

ALPACA_API_KEY = os.getenv("ALPACA_API_KEY")
ALPACA_SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")
ALPACA_BASE_URL = os.getenv("ALPACA_BASE_URL")

In [2]:
from blankly import Strategy, StrategyState, Interface
from blankly import Alpaca
from blankly.indicators import sma
from arima_model import ArimaStrategy

TICKERS = ["NWS", "NWSA"]
BENCHMARK = "SPY"
TRADE_ON = "open"
CARRY_FORWARD_PREVIOUS_DECISION = True
PCT_BALANCE_TO_TRADE = 0.3

temp = []


def init(symbols, state: StrategyState):
    # initialize this once and store it into state
    variables = state.variables

    # Download price data to give context to the algo
    # This gets the past 150 data points prior to the start of the backtest
    # Data kept as a deque to reduce memory usage
    # keys: ['high', 'low', 'volume', 'close', 'open', 'time']
    variables["x_history"] = state.interface.history(
        symbols[0], to=250, return_as="deque"
    )[TRADE_ON]
    variables["y_history"] = state.interface.history(
        symbols[1], to=250, return_as="deque"
    )[TRADE_ON]

    variables["model"] = ArimaStrategy(variables["x_history"], variables["y_history"])

    variables["prev_long_decision"] = (0, 0)
    variables["prev_short_decision"] = (0, 0)


def price_event(prices, symbols, state: StrategyState):
    # print(prices)
    # print(symbols)
    # return
    interface: Interface = state.interface
    variables = state.variables
    model: ArimaStrategy = variables["model"]
    x_data_point = prices[symbols[0]]
    y_data_point = prices[symbols[1]]

    # make a decision to buy, sell, or hold
    # returns decision: 1 for long, -1 for short, 0 for close position
    pred_beta, long_decision, short_decision = model.make_decision(
        x_data_point, y_data_point
    )

    # if we are carrying forward the previous decision, then we need to check if the current decision is None
    if CARRY_FORWARD_PREVIOUS_DECISION:
        if long_decision == (None, None):
            long_decision = variables["prev_long_decision"]
        if short_decision == (None, None):
            short_decision = variables["prev_short_decision"]
    else:
        if long_decision == (None, None):
            long_decision = (0, 0)
        if short_decision == (None, None):
            short_decision = (0, 0)

    variables["prev_long_decision"] = long_decision
    variables["prev_short_decision"] = short_decision

    decision = (
        long_decision[0] + short_decision[0],
        long_decision[1] + short_decision[1],
    )

    cash_available = interface.cash * PCT_BALANCE_TO_TRADE
    curr_x_unit = interface.account[symbols[0]].available
    curr_y_unit = interface.account[symbols[1]].available

    # indicates whether we are buying or selling
    sign_x = decision[0]
    sign_y = decision[1]

    target_y_unit = (cash_available // y_data_point) * sign_y
    target_x_unit = abs(target_y_unit) * pred_beta * sign_x

    # calculate the difference between the current and target units
    x_diff = target_x_unit - curr_x_unit
    x_diff = blankly.trunc(x_diff, 9)
    y_diff = target_y_unit - curr_y_unit
    y_diff = blankly.trunc(y_diff, 9)

    if x_diff > 0:
        interface.market_order(symbols[0], "buy", x_diff)
    elif x_diff < 0:
        interface.market_order(symbols[0], "sell", abs(x_diff))

    if y_diff > 0:
        interface.market_order(symbols[1], "buy", y_diff)
    elif y_diff < 0:
        interface.market_order(symbols[1], "sell", abs(y_diff))

    # update model
    model.update(x_data_point, y_data_point)


def doNothing(**kwargs):
    return


alpaca = Alpaca()
s = Strategy(alpaca)
# [x, y]
s.add_prices(BENCHMARK, resolution="1d", to="3M")
s.add_arbitrage_event(price_event, TICKERS, resolution="1d", init=init)
res = s.backtest(
    initial_values={"USD": 10000},
    to="3M",
    use_price="open",
    show_progress=True,
    risk_free_return_rate=0.04,
    benchmark_symbol=BENCHMARK,
)

INFO: No portfolio name to load specified, defaulting to the first in the file: (alpaca_key). This is fine if there is only one portfolio in use.



Backtesting...
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-1368.498, Time=0.04 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-1305.186, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-1351.778, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-1370.058, Time=0.01 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-1307.157, Time=0.01 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-1368.673, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-1365.276, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=-1371.639, Time=0.02 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=-1369.945, Time=0.02 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=-1370.315, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=-1353.690, Time=0.01 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=-1367.705, Time=0.03 sec

Best model:  ARIMA(0,1,1)(0,0,0)[0]          
Total fit time: 0.219 seconds
Progress: [##########] 100% Done...


/Users/juansebastian/Code/Quant/Alpha/Mean-Reversion-Pairs-Trading/venv/lib/python3.9/site-packages/numpy/matrixlib/defmatrix.py:145: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = N.array(data, dtype=dtype, copy=copy)


In [3]:
print(res.history.keys())

dict_keys(['SPY', 'NWS', 'NWSA'])
